# Model testing

- It's just a testing ground for the model. One model is loaded and a `cmc.test_model` is applied on it.
- As a result confusion matrix, ROC, AUROC, F1, PR curves will be generated and saved to the model's folder

In [1]:
import os
from pathlib import Path
import re
import time 
from datetime import datetime
import json

import matplotlib.pyplot as plt
import torch
from torch.optim import lr_scheduler
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import pytorch_lightning as pl

import alt_models as am


In [4]:
signal_name = 'mc'
architecture = 'InceptionTime'
signal_window = 320
sampling_freq = 300
batch_size = 256
num_workers = 32
num_epochs = 16
dpoints_in_future = 160
learning_rate_max = 0.01
num_classes=2
comment_for_model_name = ''
exponential_elm_decay=False
use_ELMs=True
only_ELMs=False
no_L_mode=True

random_seed = 42

if signal_name == 'mc':
    in_channels = 4
else:
    in_channels = 1

In [5]:
pl.seed_everything(random_seed)
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
path = Path(os.getcwd())
comment_for_model_name = architecture  + comment_for_model_name
    
# Load data
shot_usage = pd.read_csv(f'{path}/data/shot_usage.csv')
shot_for_alt = shot_usage[shot_usage['used_for_alt']]
shot_numbers = shot_for_alt['shot']
shots_for_testing = shot_for_alt[shot_for_alt['used_as'] == 'test']['shot']
shots_for_validation = shot_for_alt[shot_for_alt['used_as'] == 'val']['shot']
shots_for_training = shot_for_alt[shot_for_alt['used_as'] == 'train']['shot']
shot_df, test_df, val_df, train_df = am.split_df(path, shot_numbers, 
                                                 shots_for_training, 
                                                 shots_for_testing, 
                                                 shots_for_validation, 
                                                 use_ELMs=use_ELMs, 
                                                 signal_name=signal_name, 
                                                 sampling_freq=sampling_freq, 
                                                 only_ELMs=only_ELMs,
                                                 no_L_mode=no_L_mode,
                                                 exponential_elm_decay=exponential_elm_decay)

Seed set to 42


In [6]:
test_dataloader = am.get_dloader(test_df, batch_size=batch_size, 
                                balance_data=True, shuffle=False, 
                                signal_window=signal_window,
                                signal_name=signal_name,
                                num_workers=num_workers)

/compass/Shared/Users/bogdanov/.venv/lib/python3.8/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [19]:
len(a[a==0])

124

### Load the model

In [20]:
pretrained_model = am.select_model_architecture(architecture=architecture, window=signal_window, 
                                               num_classes=2, in_channels=in_channels)

#Which model to test?
model_path = f'{path}/runs/24-04-15, 19-22-16 InceptionTime ELMs only, 4 mirnov_coils, 320 window, 300 sampling_freq'

pretrained_model.load_state_dict(torch.load(f'{model_path}/model.pt'))
pretrained_model.eval()
pretrained_model.to(device)



Sequential(
  (0): Reshape()
  (1): InceptionBlock(
    (activation): ReLU()
    (inception_1): Inception(
      (bottleneck): Conv1d(4, 32, kernel_size=(1,), stride=(1,), bias=False)
      (conv_from_bottleneck_1): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (conv_from_bottleneck_2): Conv1d(32, 32, kernel_size=(11,), stride=(1,), padding=(5,), bias=False)
      (conv_from_bottleneck_3): Conv1d(32, 32, kernel_size=(23,), stride=(1,), padding=(11,), bias=False)
      (max_pool): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
      (conv_from_maxpool): Conv1d(4, 32, kernel_size=(1,), stride=(1,), bias=False)
      (batch_norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
    )
    (inception_2): Inception(
      (bottleneck): Conv1d(128, 32, kernel_size=(1,), stride=(1,), bias=False)
      (conv_from_bottleneck_1): Conv1d(32, 32, kernel_size=(5,), stride=(1,), pad

### Test model on unbalanced test dset

In [21]:
import confinement_mode_classifier as cmc
metrics = cmc.test_model(model_path, 
                         pretrained_model, 
                         test_dataloader, 
                         max_batch=0, 
                         comment='last_epoch',
                         signal_name=signal_name,
                         num_classes=num_classes)



Processing batches: 100%|██████████| 3664/3664 [00:52<00:00, 70.26it/s]


Processing metrics...


### Here the model is tested on individual shots (generates time-confidence graph)

In [22]:
shots_for_testing_list = list(map(int,shots_for_testing))
img_path = cmc.per_shot_test(path=model_path, 
                             shots=shots_for_testing_list, 
                             results_df=metrics['prediction_df'],
                             num_classes=num_classes)
plt.close()

100%|██████████| 12/12 [00:10<00:00,  1.10it/s]


In [25]:
metrics['pr_roc_curves']['pr_auc']

0.7808891